# RFC obtaintion (validation and cleaning)
The goal is to read a csv file with teo columns ("RFC" and "RAZON"), then to validate RFCs, to clean column "RAZON" by applying Regex techniques and to add column "Persona" if the RFC corresponds to a "fisica" or "moral" person from one csv file. The cleaned DataFrame is saved as "NuevoRFC.csv".

In [2]:
# Import libraries
import pandas as pd
import re
from typing import Optional
import re
import spacy # Recommended for Spanish
from rapidfuzz import fuzz

## Read non-processed csv file

In [3]:
csv_file = 'prueba.csv'
column_names = ['RFC','RAZON']
df = pd.read_csv(csv_file, sep=',', header=None, names=column_names)
df.head(5)

,RFC,RAZON
0,AAA08091161A,"APOYANDO A ANGELITOS CON AUTISMO, A. C."
1,AAA081021QD9,APOLINAR ASOCIADOS CONSULTORIA Y SERVICIOS SC
2,AAA090601GY4,DE ALBA & ASOCIADOS FIRMA LEGAL S.C.
3,AAA090924HJ4,"ARGUELLES, ALVAREZ & ASOCIADOS SA DE CV"
4,AAA1002249W6,ADAIR ALONSO ARQUITECTOS SA DE CV


In [10]:
initial_rows = df.shape[0]
print(f"El archivo tiene {initial_rows} filas.")

El archivo tiene 10 filas.


In [4]:
# Delete all rows with null values in some columns
df = df.dropna()

# Change column types
df[column_names] = df[column_names].astype('str')

## Validate RFC

In [5]:
# Regular expressions
RFC_FISICA_REGEX = re.compile(
    r"^[A-ZÑ&]{4}"
    r"\d{2}(0[1-9]|1[0-2])"
    r"(0[1-9]|[12]\d|3[01])"
    r"[A-Z0-9]{3}$"
)

RFC_MORAL_REGEX = re.compile(
    r"^[A-ZÑ&]{3}"
    r"\d{2}(0[1-9]|1[0-2])"
    r"(0[1-9]|[12]\d|3[01])"
    r"[A-Z0-9]{3}$"
)

def normalize_rfc(rfc: str) -> str:
    """Limpia espacios y convierte a mayúsculas"""
    return rfc.strip().strip(".").strip().upper()

# Normalize RFC
df['RFC'] = df['RFC'].apply(normalize_rfc)

In [6]:
def is_persona_fisica(rfc: str) -> bool:
    """Verifica si el RFC corresponde a persona física"""
    return bool(RFC_FISICA_REGEX.match(rfc))


def is_persona_moral(rfc: str) -> bool:
    """Verifica si el RFC corresponde a persona moral"""
    return bool(RFC_MORAL_REGEX.match(rfc))


def get_rfc_type(rfc: str) -> Optional[str]:
    """Validate RFC"""

    if is_persona_fisica(rfc):
        return "FISICA"
    if is_persona_moral(rfc):
        return "MORAL"

    return None

# Validate RFC
df['PERSONA'] = df['RFC'].apply(get_rfc_type)

# Filter
df = df[(df['PERSONA'] == 'FISICA') | (df['PERSONA'] == 'MORAL')]

df.sample(5)

,RFC,RAZON,PERSONA
4,AAA1002249W6,ADAIR ALONSO ARQUITECTOS SA DE CV,MORAL
8,AAA101112AM4,APLICACIONES ADMINISTRATIVAS AVM S.C.,MORAL
1,AAA081021QD9,APOLINAR ASOCIADOS CONSULTORIA Y SERVICIOS SC,MORAL
9,AAA1012228L1,AGUILAR ABASCAL Y ASOCIADOS SC,MORAL
2,AAA090601GY4,DE ALBA & ASOCIADOS FIRMA LEGAL S.C.,MORAL


In [7]:
final_rows = df.shape[0]
print(f"El archivo tiene {final_rows} RFCs válidos.")

El archivo tiene 10 RFCs válidos.


## Column "RAZON" preprocessing

In [9]:
def normalize_text(text: str, pattern: str, new_value: str) -> str:
    """Delete trash from text"""
    text = str(text).strip(".,; ").replace(",", "").replace(";", "").replace(".", "").upper()
    text = re.sub(pattern, new_value, text)
    text = re.sub(r'[/\\-]', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()

# Patterns like S.A., C.V.
norm_rules = {
    r'\bS\.?\s?A\.? ': "SA ",
    r'\bS\.?\s?A\.?\b': "SA",
    r'\bC\.?\s?V\.?\b': "CV",
    r'\bS\.?\s?C\.?\b': "SC",
    r'\bS\.?\s?A\.?P\.?\s?I\.?\s?\b': "SAPI ",
    r'\bA\.?\s?C\.? ': "AC ",
    r'\bA\.?\s?C\.?\b': "AC",
}

# Normalize "RAZON"
df['NOMBRE'] = df[column_names[1]]

for pattern, replacement in norm_rules.items():
    df['NOMBRE'] = df['NOMBRE'].apply(
        normalize_text, 
        pattern=pattern, 
        new_value=replacement
    )

df.sample(5)

,RFC,RAZON,PERSONA,NOMBRE
0,AAA08091161A,"APOYANDO A ANGELITOS CON AUTISMO, A. C.",MORAL,APOYANDO A ANGELITOS CON AUTISMO AC
2,AAA090601GY4,DE ALBA & ASOCIADOS FIRMA LEGAL S.C.,MORAL,DE ALBA & ASOCIADOS FIRMA LEGAL SC
9,AAA1012228L1,AGUILAR ABASCAL Y ASOCIADOS SC,MORAL,AGUILAR ABASCAL Y ASOCIADOS SC
4,AAA1002249W6,ADAIR ALONSO ARQUITECTOS SA DE CV,MORAL,ADAIR ALONSO ARQUITECTOS SA DE CV
5,AAA100303L51,"INGENIOS SANTOS, S.A. DE C.V.",MORAL,INGENIOS SANTOS SA DE CV


In [10]:
def lemmatize_text(text: str) ->str:
    """Apply tokenization and lemmatization to the text"""
    # Load the model
    nlp = spacy.load("es_core_news_sm")

    # Process text
    doc = nlp(text.lower())

    # Tokenization and lemmatization
    # tokens = [token.text for token in doc]
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas).upper()

# Lemmatization
df['LEMMA_SPA'] = df['NOMBRE'].apply(lemmatize_text)

df[['NOMBRE', 'LEMMA_SPA']].head(5)

,NOMBRE,LEMMA_SPA
0,APOYANDO A ANGELITOS CON AUTISMO AC,APOYAR A ANGELITO CON AUTISMO AC
1,APOLINAR ASOCIADOS CONSULTORIA Y SERVICIOS SC,APOLINAR ASOCIADO CONSULTORIO Y SERVICIO SC
2,DE ALBA & ASOCIADOS FIRMA LEGAL SC,DE ALBA & ASOCIADOS FIRMA LEGAL SC
3,ARGUELLES ALVAREZ & ASOCIADOS SA DE CV,ARGUELL ALVAREZ & ASOCIADOS SA DE CV
4,ADAIR ALONSO ARQUITECTOS SA DE CV,ADAIR ALONSO ARQUITECTOS SA DE CV


## Sort values

In [11]:
# Sort by 'RFC'
df = df.sort_values(by="RFC", ascending=True).reset_index(drop=True)

df.head(10)

,RFC,RAZON,PERSONA,NOMBRE,LEMMA_SPA
0,AAA08091161A,"APOYANDO A ANGELITOS CON AUTISMO, A. C.",MORAL,APOYANDO A ANGELITOS CON AUTISMO AC,APOYAR A ANGELITO CON AUTISMO AC
1,AAA081021QD9,APOLINAR ASOCIADOS CONSULTORIA Y SERVICIOS SC,MORAL,APOLINAR ASOCIADOS CONSULTORIA Y SERVICIOS SC,APOLINAR ASOCIADO CONSULTORIO Y SERVICIO SC
2,AAA090601GY4,DE ALBA & ASOCIADOS FIRMA LEGAL S.C.,MORAL,DE ALBA & ASOCIADOS FIRMA LEGAL SC,DE ALBA & ASOCIADOS FIRMA LEGAL SC
3,AAA090924HJ4,"ARGUELLES, ALVAREZ & ASOCIADOS SA DE CV",MORAL,ARGUELLES ALVAREZ & ASOCIADOS SA DE CV,ARGUELL ALVAREZ & ASOCIADOS SA DE CV
4,AAA1002249W6,ADAIR ALONSO ARQUITECTOS SA DE CV,MORAL,ADAIR ALONSO ARQUITECTOS SA DE CV,ADAIR ALONSO ARQUITECTOS SA DE CV
5,AAA100303L51,"INGENIOS SANTOS, S.A. DE C.V.",MORAL,INGENIOS SANTOS SA DE CV,INGENIO SANTO SA DE CV
6,AAA100426EG5,ALONSO Y AUDITORES ASOCIADOS SC,MORAL,ALONSO Y AUDITORES ASOCIADOS SC,ALONSO Y AUDITOR ASOCIADO SC
7,AAA100705VD8,ANBORZ ASOCIADOS SC,MORAL,ANBORZ ASOCIADOS SC,ANBORZ ASOCIADO SC
8,AAA101112AM4,APLICACIONES ADMINISTRATIVAS AVM S.C.,MORAL,APLICACIONES ADMINISTRATIVAS AVM SC,APLICACIÓN ADMINISTRATIVO AVM SC
9,AAA1012228L1,AGUILAR ABASCAL Y ASOCIADOS SC,MORAL,AGUILAR ABASCAL Y ASOCIADOS SC,AGUILAR ABASCAL Y ASOCIADO SC


In [12]:
# Reorder columns
dataFrame = df[['RFC','RAZON', 'NOMBRE', 'LEMMA_SPA', 'PERSONA']]

# Save DataFrame
file_name = f"NuevosRFC_{csv_file.split(".")[0]}"
dataFrame.to_csv(f'{file_name}.csv', encoding='utf-8', index=False)

## Spacy Models (Recommended for Spanish)
Download the corresponding model from "https://github.com/explosion/spacy-models/releases/tag/es_core_news_sm-3.8.0" ("es_core_news_sm-3.8.0-py3-none-any.whl" file) manually and paste it in this project. After that, run:
```
%pip install "es_core_news_sm-3.8.0-py3-none-any.whl"
```
Or run:
```
python -m spacy download es_core_news_sm
```


In [21]:
# %pip install "es_core_news_sm-3.8.0-py3-none-any.whl"

In [22]:
import spacy # Recommended for Spanish

In [18]:
from thefuzz import fuzz

def calcular_similitud_razon_social(cadena1, cadena2):
    """
    Compara dos razones sociales y retorna un puntaje de 0 a 100.
    Utiliza partial_token_sort_ratio para manejar variaciones en el orden
    y nombres que contienen subcadenas parciales.
    """
    # Validamos que ambos sean strings
    if not isinstance(cadena1, str) or not isinstance(cadena2, str):
        return 0
    
    # Aplicamos el método de comparación difusa
    puntuacion = fuzz.partial_token_sort_ratio(cadena1, cadena2)
    
    return puntuacion

In [22]:
text1 = 'ABARROTES LA PASADITA'
text2 = 'LA PASADITA'
# 1. Cargas el modelo en español
nlp = spacy.load("es_core_news_sm")

# 2. Procesas el texto
doc1 = nlp(text1.lower())
doc2 = nlp(text2.lower())

# 3. Tokenización y Lematización automática
tokens = [token.text for token in doc1]
lemmas1 = [token.lemma_ for token in doc1]

limpio1 = ' '.join(lemmas1)
print(limpio1)

tokens = [token.text for token in doc2]
lemmas2 = [token.lemma_ for token in doc2]

limpio2 = ' '.join(lemmas2)
print(limpio2)

score = calcular_similitud_razon_social(limpio1, limpio2)
print(f"La calificación de similitud es: {score}")

abarrot el pasadita
el pasadita
La calificación de similitud es: 100


### Ejemplos
BBVA BANCOMER	        BANCOMER BBVA S.A.

COCA COLA FEMSA         FEMSA COCA-COLA

ABARROTES LA PASADITA	LA PASADITA

AGROINDUSTRIAS APLICACIONES ADMINISTRATIVAS AVM SC      ADMINISTRATIVAS APLICACIONES AVM SC

In [23]:
import pandas as pd
from thefuzz import fuzz

def ordenar_por_similitud(df, columna_datos, cadena_referencia):
    """
    Calcula la similitud de cada fila contra una cadena de referencia,
    añade la puntuación al DataFrame y lo ordena de mayor a menor.
    """
    
    # 1. Creamos una función interna para aplicar el cálculo
    def aplicar_fuzz(texto):
        # Aseguramos que el texto sea string para evitar errores con NaNs
        return fuzz.partial_token_sort_ratio(str(texto), cadena_referencia)

    # 2. Creamos la nueva columna con las calificaciones
    df['similitud_score'] = df[columna_datos].apply(aplicar_fuzz)

    # 3. Ordenamos de mayor a menor (las más parecidas primero)
    df = df.sort_values(by='similitud_score', ascending=False).reset_index(drop=True)

    return df

# --- Ejemplo de uso ---
data = {
    'RFC': ['ABC123', 'DEF456', 'GHI789', 'JKL012'],
    'RAZON_SOCIAL': ['MC SA DE CV', 'S.A. DE C.V. MC', 'MADERAS Y CONCRETOS', 'WALMART MEXICO']
}

df_ejemplo = pd.DataFrame(data)

# Supongamos que buscamos todo lo parecido a "MC"
df_ordenado = ordenar_por_similitud(df_ejemplo, 'RAZON_SOCIAL', 'MC')

print(df_ordenado)

      RFC         RAZON_SOCIAL  similitud_score
0  ABC123          MC SA DE CV              100
1  DEF456      S.A. DE C.V. MC              100
2  GHI789  MADERAS Y CONCRETOS               67
3  JKL012       WALMART MEXICO               67


In [ ]:
def get_unique_names(column: pd.Series) -> pd.DataFrame:
    """Extract unique values in a column and return a DataFrame of it"""
    uniques = column.unique()
    
    # Create DataFrame
    df_sample = pd.DataFrame({'UNIQUE': uniques})
    
    # Reset index
    df_sample['ID'] = df_sample.index
    
    return df_sample

df_uniques = get_unique_names(df['LEMMA_SPA'])

df_uniques.head()

,UNIQUE,ID
0,APOYAR A ANGELITO CON AUTISMO AC,0
1,APOLINAR ASOCIADO CONSULTORIO Y SERVICIO SC,1
2,DE ALBA & ASOCIADOS FIRMA LEGAL SC,2
3,ARGUELL ALVAREZ & ASOCIADOS SA DE CV,3
4,ADAIR ALONSO ARQUITECTOS SA DE CV,4
